In [1]:
# This notebook is used to extract a Job Description from a Linkedin Job's ID

# Import the necessary libraries
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup as bs4
from time import sleep


In [3]:
DRIVER_PATH = '../driver/mac/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.maximize_window()

OSError: [Errno 8] Exec format error: '../driver/linux/chromedriver_linux64/chromedriver'

In [3]:
def login():
      sleep(2)
      url = 'https://www.linkedin.com/login'
      driver.get(url)
      print('- Finish initializing a driver')
      sleep(2)

      # Task 1.2: Import username and password
      credential = open('credentials.txt')
      line = credential.readlines()
      username = line[0]
      password = line[1]
      print('- Finish importing the login credentials')
      sleep(1)

      # Task 1.2: Key in login credentials
      email_field = driver.find_element_by_id('username')
      email_field.send_keys(username)
      print('- Finish keying in email')
      sleep(3)

      password_field = driver.find_element_by_name('session_password')
      password_field.send_keys(password)
      print('- Finish keying in password')
      sleep(3)

      # Task 1.2: Click the Login button
      signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
      signin_field.click()
      sleep(3)

      print('- Finish Task 1: Login to Linkedin')

      # Task 1.3: Check if the login is successful
login()

- Finish initializing a driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task 1: Login to Linkedin


In [8]:
def extract_JD(job_id=''):
        # Task 2.1: Access the job page
        url = 'https://www.linkedin.com/jobs/view/' + job_id
        driver.get(url)
        sleep(2)

        button_showmore = driver.find_elements_by_css_selector("[aria-label='Click to see more description'")
        button_showmore[0].click()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Task 2.2: Extract the Job Description
        jobTitle = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > h1').text
        location = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt2 > span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__bullet').text
        company = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt2 > span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__company-name').text
        type = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt5.mb2 > ul > li:nth-child(1) > span').text
        # status = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt5.mb2 > ul > li:nth-child(4) > span').text
        description =  driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div.jobs-box--fadein.jobs-box--full-width.jobs-box--with-cta-large.jobs-description.artdeco-card.mb4.jobs-poster--is-expanded-redesign > article').text
        
        print('Job Title: ', jobTitle)
        print('Location: ', location)
        print('Company: ', company)
        print("Work type:", type)
        # print("Job status: ", status)
        print('Description: ',description)

        # Task 2.3: Save the Job Description to a csv file
    

In [9]:
extract_JD('2949802590')

AttributeError: 'NoneType' object has no attribute 'text'